In [1]:
def plot_pass_network(match_id,team_name,min_passes=5):
    from statsbombpy import sb
    import pandas as pd
    import networkx as nx
    from mplsoccer import Pitch
    import matplotlib.pyplot as plt
    #Load event data
    events = sb.events(match_id=match_id)
    passes = events[
        (events['type']=='Pass') & 
        (events['team']==team_name) &
        (events['pass_type']!='Throw-In') & 
        (events['pass_outcome'].isna())
    ]

    passes = passes[['player', 'pass_recipient', 'location', 'pass_end_location']].dropna()
    passes[['x', 'y']] = pd.DataFrame(passes['location'].tolist(), index=passes.index)
    passes[['end_x', 'end_y']] = pd.DataFrame(passes['pass_end_location'].tolist(), index=passes.index)

    # Network graph
    G = nx.DiGraph()
    for _, row in passes.iterrows():
        passer = row['player']
        recipient = row['pass_recipient']
        if G.has_edge(passer, recipient):
            G[passer][recipient]['weight'] += 1
        else:
            G.add_edge(passer, recipient, weight=1)

    # Remove weak edges
    G.remove_edges_from([(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_passes])

    #Compute centrality
    centrality = nx.degree_centrality(G)
    
    avg_locs = passes.groupby('player')[['x', 'y']].mean()

    pitch = Pitch(pitch_color='grass',line_color='white',stripe=True)
    fig, ax = pitch.draw(figsize=(10, 7))

    for player, (x, y) in avg_locs.iterrows():
        size = centrality.get(player, 0) * 2000
        ax.scatter(x, y, s=size, color='skyblue', edgecolor='black', zorder=3)
        ax.text(x, y, player.split()[0], fontsize=9, ha='center', va='center', zorder=4)

    for u, v, d in G.edges(data=True):
        if u in avg_locs.index and v in avg_locs.index:
            x1, y1 = avg_locs.loc[u]
            x2, y2 = avg_locs.loc[v]
            pitch.arrows(x1,y1,x2,y2,ax=ax,color='pink',linewidth=0.1)

    teams = events['team'].unique()

    opp_team = [t for t in teams if t!=team_name][0]

    ax.set_title(f"{team_name} Pass Network vs. {opp_team}", fontsize=16)
    plt.show()
    plt.close(fig)

    # Print top 5 most central players
    top_5 = sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:5]
    print(f"\nTop 5 players by degree centrality vs. {opp_team}:")
    for rank, (player, score) in enumerate(top_5, 1):
        print(f"{rank}. {player}: {score:.3f}")

    return centrality

In [2]:
def compute_xg_chain_and_buildup(events, shots):
    from collections import defaultdict
    import pandas as pd
    xg_chain = defaultdict(float)
    xg_buildup = defaultdict(float)

    # Precompute possession-level xG
    possession_xg = shots.groupby('possession')['shot_statsbomb_xg'].sum().to_dict()

    for pid in possession_xg:
        xg = possession_xg[pid]
        if xg <= 0:
            continue

        shot_event = shots[shots['possession'] == pid]
        if shot_event.empty:
            continue

        team_in_possession = shot_event['possession_team'].iloc[0]

        possession_events = events[
            (events['possession'] == pid) &
            (events['team'] == team_in_possession)
        ]

        valid_players = possession_events['player'].dropna().tolist()
        if not valid_players:
            continue

        shooters = set(shot_event['player'].dropna())
        assisters = set(events[
            (events['possession'] == pid) &
            (events['pass_shot_assist'] == True)
        ]['player'])

        xg_share = xg / len(valid_players)

        for player in valid_players:
            xg_chain[player] += xg_share
            if player not in shooters and player not in assisters:
                xg_buildup[player] += xg_share

    #xg_chain_df = pd.DataFrame.from_dict(xg_chain, orient='index', columns=['xGChain'])
    #xg_buildup_df = pd.DataFrame.from_dict(xg_buildup, orient='index', columns=['xGBuildup'])

    # Create player → team mapping
    player_team_map = events[['player', 'team']].dropna().drop_duplicates()
    player_team_dict = player_team_map.set_index('player')['team'].to_dict()
    
    
    xg_df = pd.DataFrame({
        'team': [player_team_dict.get(p, 'Unknown') for p in xg_chain],
        'xGChain': [xg_chain[p] for p in xg_chain],
        'xGBuildup': [xg_buildup.get(p, 0) for p in xg_chain]
    })

    xg_df.index = list(xg_chain.keys())

    xg_df = xg_df.sort_values('xGChain', ascending=False)
    
    return xg_df